In [1]:
import tensorflow as tf
from pydst.rate_limiters import RateLimited 
from pydst.dataproviders import DataProvider
from IPython.display import display, HTML
import datetime
import os
import numpy as np

In [2]:
def show_graph(graph_def, frame_size=(900, 600)):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:{height}px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(height=frame_size[1], data=repr(str(graph_def)), id='graph'+timestamp)
    iframe = """
        <iframe seamless style="width:{width}px;height:{height}px;border:0" srcdoc="{src}"></iframe>
    """.format(width=frame_size[0], height=frame_size[1] + 20, src=code.replace('"', '&quot;'))
    display(HTML(iframe))

In [11]:
def dense(name, inputs, units, fn, kn_init)
    with tf.variable_scope(name):
        output = tf.layers.dense(inputs=inputs, units=units, name=name, activation=fn, 
                                 kernel_initializer=tf.truncated_normal_initializer(stddev=(2./(inputs.shape[1]+units))**0.5)


def mlp_graph(which_set, rng=None):
    
    graph = tf.Graph()
    outputs = {}
    errors = {}
    
    with graph.as_default():

        with tf.device('/cpu:0'):
            dataprovider = DataProvider(graph=graph, which_set=which_set, batch_size=10, 
                                                                            down_sample=20, target_size=30,rng=rng)
            inputs, targets = dataprovider.get_data()

        name = 'layer-1'
        with tf.variable_scope(name):
            outputs[name] = tf.layers.dense(inputs=inputs, units=4000, name=name, activation=tf.nn.relu,
                                kernel_initializer=tf.truncated_normal_initializer(stddev=(2./(inputs.shape[1]+units))**0.5)
                                            

        name = 'layer-2'
        with tf.variable_scope(name):
            outputs[name] = tf.layers.dense(inputs=outputs['layer-1'], units=2000, name=name, activation=tf.nn.relu)

        name = 'layer-3'
        with tf.variable_scope(name):
            outputs[name] = tf.layers.dense(inputs=outputs['layer-2'], units=1000, name=name, activation=tf.nn.relu)
            
        name = 'layer-4'
        with tf.variable_scope(name):
            outputs[name] = tf.layers.dense(inputs=outputs['layer-3'], units=100, name=name, activation=tf.nn.relu)
            
        name = 'layer-5'
        with tf.variable_scope(name):
            outputs[name] = tf.layers.dense(inputs=outputs['layer-4'], units=30, name=name, activation=tf.nn.relu)

        name = 'error-l5'
        with tf.variable_scope(name):
            errors[name] = tf.losses.sigmoid_cross_entropy(multi_class_labels=targets, logits=outputs['layer-5'])

        train_step = tf.train.AdamOptimizer().minimize(errors[name])

        tf.summary.scalar(name, errors[name])
        summary_op = tf.summary.merge_all()
        saver = tf.train.Saver()
        
        sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=8))
        
    return sess, graph, summary_op, train_step, dataprovider, saver, inputs, targets

In [12]:
with tf.variable_scope("model_t1") as scope:
    [train_sess, train_graph, train_summary, train_step, trainData, trainSaver, 
                                                                tinputs, ttargets] = mlp_graph('train')
    
#show_graph(train_graph)

timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
writer = tf.summary.FileWriter(os.path.join('logs', 'dense_exp', timestamp, 'train'), graph=train_graph)
with tf.device('/cpu:1'):
    with train_graph.as_default():

        train_sess.run(tf.global_variables_initializer())
        trainData.enable(train_sess)

        max_iters = trainData.max_num_batches()

        print('Starting iterations')
        for epoch in range(30):

            run_error = 0.
            run_acc = 0.
            print('Epoch: {}'.format(epoch))
            for idx in range(max_iters):

                [_, summary_results] = train_sess.run([train_step, train_summary])          
                writer.add_summary(summary_results, epoch * max_iters + idx)

        trainData.disable(train_sess)

Starting iterations
Epoch: 0
Epoch: 1
Epoch: 2


KeyboardInterrupt: 